<a href="https://colab.research.google.com/github/engbasem/ML-Taining-Notebooks/blob/main/Mounting_GoogleDrive_Read_write_SaveProgress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mounting Google drive to Colab instance**

[R1] https://machinelearningmastery.com/google-colab-for-machine-learning-projects/

If you prefer storing all your files in Google drive, you can easily mount it onto the current instance of Colab. This will enable you to access all the datasets and files that are stored in your drive.

There are a couple of ways you can do this:

* You can click on the 'drive' icon in the 'Files' tab and follow the prompts on the screen.

*  You can also mount your Google drive to the current Colab instance, by running the following lines of code in a code cell in your notebook.

In [2]:
import os
from google.colab import drive
 
MOUNTPOINT = "/content/gdrive"
DATADIR = os.path.join(MOUNTPOINT, "MyDrive")
drive.mount(MOUNTPOINT)

Mounted at /content/gdrive


Then, to write a file to our Google Drive, we can do the following. This code snippet writes Hello World! to a test.txt file in the top level of your Google Drive.

In [3]:
# writes directly to google drive
with open(f"{DATADIR}/test.txt", "w") as outfile:
    outfile.write("Hello World!")

Similarly, we can read from a file in our Google Drive as well by using:

In [4]:
with open(f"{DATADIR}/test.txt", "r") as infile:
    file_data = infile.read()
    print(file_data)

Hello World!


**Saving Model Progress on Google Drive**

Google Colab is probably the easiest way to give us powerful GPU resources for your machine learning project. But in the free version of Colab, Google limits the time we can use our Colab notebook in each session. Our kernel may terminate for no reason. We can restart our notebook and continue our work, but we may lose everything in the memory. This is a problem if we need to train our model for a long time. Our Colab instance may terminate before the training is completed.

**Mounting Google Drive and using Keras ModelCheckpoint callback**

we can save our model progress on Google Drive. This is particularly useful to work around Colab timeouts. It is more lenient for paid Pro and Pro+ users, but there is always a chance that our model training terminates midway at random times. 

**It is valuable if we don’t want to lose our partially trained model.**

In [5]:
# Example using LENet5 on mnist dataset
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Conv2D, Flatten, MaxPool2D
from keras.models import Model
 
class LeNet5(tf.keras.Model):
  def __init__(self):
    super(LeNet5, self).__init__()
    #creating layers in initializer
    self.conv1 = Conv2D(filters=6, kernel_size=(5,5), padding="same", activation="relu")
    self.max_pool2x2 = MaxPool2D(pool_size=(2,2))
    self.conv2 = Conv2D(filters=16, kernel_size=(5,5), padding="same", activation="relu")
    self.flatten = Flatten()
    self.fc1 = Dense(units=120, activation="relu")
    self.fc2 = Dense(units=84, activation="relu")
    self.fc3=Dense(units=10, activation="softmax")
  def call(self, input_tensor):
    conv1 = self.conv1(input_tensor)
    maxpool1 = self.max_pool2x2(conv1)
    conv2 = self.conv2(maxpool1)
    maxpool2 = self.max_pool2x2(conv2)
    flatten = self.flatten(maxpool2)
    fc1 = self.fc1(flatten)
    fc2 = self.fc2(fc1)
    fc3 = self.fc3(fc2)
    return fc3

In [10]:
# you must mount GoogleDrive first
import os
from google.colab import drive
MOUNTPOINT = "/content/drive"
DATADIR = os.path.join(MOUNTPOINT, "MyDrive")
drive.mount(MOUNTPOINT)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
#Afterward, we declare the Callback to save our checkpoint model to Google Drive.
import tensorflow as tf
 
checkpoint_path = DATADIR + "/checkpoints/cp-epoch-{epoch}.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [12]:
# Next, we begin training on the MNIST dataset with the checkpoint callbacks
# to ensure we can resume at the last epoch should our Colab session time out:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Conv2D, Flatten, MaxPool2D
from keras.models import Model
 
mnist_digits = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist_digits.load_data()
 
input_layer = Input(shape=(28,28,1))
model = LeNet5()(input_layer)
model = Model(inputs=input_layer, outputs=model)
model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics="acc")
model.fit(x=train_images, y=train_labels, batch_size=256, validation_data = [test_images, test_labels], epochs=5, callbacks=[cp_callback])

Epoch 1/5
235/235 [==============================] - ETA: 0s - loss: 0.9866 - acc: 0.8714
Epoch 1: saving model to /content/drive/MyDrive/checkpoints/cp-epoch-1.ckpt
235/235 [==============================] - 36s 151ms/step - loss: 0.9866 - acc: 0.8714 - val_loss: 0.1623 - val_acc: 0.9512
Epoch 2/5
235/235 [==============================] - ETA: 0s - loss: 0.1320 - acc: 0.9615
Epoch 2: saving model to /content/drive/MyDrive/checkpoints/cp-epoch-2.ckpt
235/235 [==============================] - 35s 150ms/step - loss: 0.1320 - acc: 0.9615 - val_loss: 0.1032 - val_acc: 0.9686
Epoch 3/5
235/235 [==============================] - ETA: 0s - loss: 0.0808 - acc: 0.9762
Epoch 3: saving model to /content/drive/MyDrive/checkpoints/cp-epoch-3.ckpt
235/235 [==============================] - 35s 151ms/step - loss: 0.0808 - acc: 0.9762 - val_loss: 0.0938 - val_acc: 0.9715
Epoch 4/5
235/235 [==============================] - ETA: 0s - loss: 0.0575 - acc: 0.9822
Epoch 4: saving model to /content/drive/

**If model training stops midway, we can continue by just recompiling the model and loading the weights, and then we can continue our training:**


In [13]:
checkpoint_path = DATADIR + "/checkpoints/cp-epoch-{epoch}.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
 
input_layer = Input(shape=(28,28,1))
model = LeNet5()(input_layer)
model = Model(inputs=input_layer, outputs=model)
model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics="acc")
 
# to resume from epoch 5 checkpoints
model.load_weights(DATADIR + "/checkpoints/cp-epoch-5.ckpt")
 
# continue training
model.fit(x=train_images, y=train_labels, batch_size=256, validation_data = [test_images, test_labels], 
          epochs=5, callbacks=[cp_callback])

Epoch 1/5
235/235 [==============================] - ETA: 0s - loss: 0.0330 - acc: 0.9891
Epoch 1: saving model to /content/drive/MyDrive/checkpoints/cp-epoch-1.ckpt
235/235 [==============================] - 36s 151ms/step - loss: 0.0330 - acc: 0.9891 - val_loss: 0.0645 - val_acc: 0.9832
Epoch 2/5
235/235 [==============================] - ETA: 0s - loss: 0.0273 - acc: 0.9914
Epoch 2: saving model to /content/drive/MyDrive/checkpoints/cp-epoch-2.ckpt
235/235 [==============================] - 36s 151ms/step - loss: 0.0273 - acc: 0.9914 - val_loss: 0.0594 - val_acc: 0.9833
Epoch 3/5
235/235 [==============================] - ETA: 0s - loss: 0.0202 - acc: 0.9933
Epoch 3: saving model to /content/drive/MyDrive/checkpoints/cp-epoch-3.ckpt
235/235 [==============================] - 36s 152ms/step - loss: 0.0202 - acc: 0.9933 - val_loss: 0.0689 - val_acc: 0.9814
Epoch 4/5
235/235 [==============================] - ETA: 0s - loss: 0.0234 - acc: 0.9916
Epoch 4: saving model to /content/drive/